In [26]:
from robingrad import Tensor, draw_dot
import robingrad.nn as nn
import torch
from sklearn.datasets import make_regression
import numpy as np
import time

### Dataset

In [27]:
X, y = make_regression(n_samples=200, n_features=5, n_targets=1, random_state=42)
print(X.shape)
print(y.shape)
print(X[0])
print(y[0:5])

(200, 5)
(200,)
[0.95927083 0.02451017 0.49799829 1.45114361 2.15318246]
[172.54436082 -52.9529574   -4.14935233 193.99958991 -66.96262773]


In [28]:
X_train = Tensor(X, requires_grad=True)
print(type(X_train), X_train.shape)
y_train = Tensor(y, requires_grad=True)
print(type(y_train), y_train.shape)

<class 'robingrad.tensor.Tensor'> (200, 5)
<class 'robingrad.tensor.Tensor'> (200,)


In [29]:
X_train[0:2].data

array([[ 0.95927083,  0.02451017,  0.4979983 ,  1.4511436 ,  2.1531825 ],
       [-0.07443343, -1.5702248 , -0.72713715, -0.24751863,  0.6206721 ]],
      dtype=float32)

In [30]:
X_train[0].shape

(5,)

### Model

In [31]:
class TinyNet:
    def __init__(self):
        self.l1 = nn.Linear(5,16)
        self.l2 = nn.Linear(16,1)
    def __call__(self, x):
        x = self.l1(x)
        x = x.relu()
        x = self.l2(x)
        return x
        
net = TinyNet()

In [32]:
res = net(X_train[0])
res

Tensor: __add__
data: 
[7.554098]
dtype: float32

In [36]:
draw_dot(res, inspect=True)

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

### Full batch gradient descent with squared error loss
Squared error (squared L2 norm):
$$ l(x,y) = L = (\hat{y}_i - y_i)^2$$

In [35]:
epochs = 100
for epoch in range(epochs):
    y_pred = []
    losses = []
    s = time.monotonic()
    for i in range(X_train.shape[0]):
        output = net(X_train[i])
        y_pred.append(output.data)
        target = y_train[i]
        loss = (output-target)**2
        losses.append(loss.data)
        loss.backward()
        for p in [net.l1.weight, net.l1.bias, net.l2.weight, net.l2.bias, net.l3.weight, net.l3.bias]:
            p.data += -3e-4*p.grad
        for p in [net.l1.weight, net.l1.bias, net.l2.weight, net.l2.bias, net.l3.weight, net.l3.bias]:
            p.grad = np.zeros_like(p.grad)
    e = time.monotonic()
    t = e - s
    loss_epoch = sum(losses)/ len(losses)
    r2 = r2_score(y, y_pred)
    if epoch % 10 == 0 or epoch == epochs-1:
        print(f"epoch: {epoch} | loss: {loss_epoch:.2f} | R2: {r2:.2f} | time: {t:.2f} sec.")
        

TypeError: unsupported operand type(s) for +: 'NoneType' and 'float'